## A/B Testing Tutorial : Python Code and Dataset 

This is a short python tutorial that takes the data provided in the attached csv file and calculates whether there is a statistically significant difference in the observed conversion rates for a treatment group and a control group. This would be done as part of the statistical analysis for A/B testing. 

In [16]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import scipy.stats as stats
import statsmodels.stats.proportion as ssp

pd.options.display.float_format = "{:.2f}".format

AB_data = pd.read_csv('/Users/emilee/Documents/data/AB_Test_data.csv')
AB_data.head()

,user_id,variation,converted
0,4681,control,1
1,9052,treatment,0
2,9579,treatment,0
3,2601,treatment,0
4,7136,control,0


In [4]:
AB_data.shape

(10000, 3)

In [5]:
## Create summary table with totals, conversions and proportions
summary = AB_data.pivot_table(values='converted', index='variation', aggfunc=np.sum)
summary['total'] = AB_data.pivot_table(values='converted', index='variation', aggfunc=lambda x: len(x))
summary['perc'] = summary['converted']/summary['total']
summary

,converted,total,perc
variation,,,
control,999,4976,0.20
treatment,1211,5024,0.24


We have 4,976 users in our control group and 5,024 in our treatment group.
Of those there were 999 conversions in the control group (20%) and 1,211 conversions in the treatment group (24%)

**Testing to see if there is a significant difference** 

In [6]:
# data wrangling, changing type to np. array
converted = np.array(summary['converted'])
group_totals = np.array(summary['total'])

In [14]:
## Calculate difference between the two groups and look at 95% CI
significance = 0.05
confidence = 1 - significance

z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)

success_a = converted[1]
success_b = converted[0]
size_a = group_totals[1]
size_b = group_totals[0]

prop_a = success_b / size_b
prop_b = success_a / size_a
var = prop_a * (1 - prop_a) / size_a + prop_b * (1 - prop_b) / size_b
se = np.sqrt(var)
    
# standard formula for the confidence interval: point-estimtate +- z * standard-error
prop_diff = prop_b - prop_a
confint = prop_diff + np.array([-1, 1]) * z * se

print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.04027932803571793
confidence interval: [0.02403373 0.05652493]


In [17]:
# See whether there is a statistically significant difference 
stat, pval = ssp.proportions_ztest(converted, group_totals)
print('p value: ', pval)
print('formated p value: ''{0:0.3f}'.format(pval))

p value:  1.2111464537151192e-06
formated p value: 0.000


**Summary**<br>
We have 4,976 users in our control group and 5,024 in our treatment group.<br>
Of those there were 999 conversions in the control group (20%) and 1,211 conversions in the treatment group (24%).<br> The point estimate difference between groups is approx. 4% with a 95% CI (2.4%, 5.7%)

For our A/B test, the null hypothesis states that there is no significant difference in the conversion rates between the control group and the treatment group.
The alternative hypothesis is that there is a significant difference.

Out statistical test has given us a p-value of approx. 0.000 which is less than the conventional cut-off of 0.05. This allows us to reject the null hypothesis and conclude that the higher conversion rate noted in the treatment group is indeed statistically significant and likely to be indicitive of a true difference in converstion rates in our full population of users. 

However we would need to consider the effect size (estimated between 2,4% and 5,7%) within the context of the business to understand whether it is a meaningful difference and whether it is a feasible ROI for the suggested product changes.